In [1]:
#import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

C:\Users\seeml\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('C:/Users/seeml/Desktop/losalamosdata.csv', error_bad_lines = False)
#print(df.target_names.unique())
df.head()

,Unnamed: 0,source_computer,destination_computer,time,source_user@domain,destination_user@domain,auth_type,longon_type,auth_orien,success/fail
0,1,C2815,C625,3721,C2815$@DOM1,C2815$@DOM1,Kerberos,Network,LogOn,Success
1,2,C14046,C754,33492,ANONYMOUS LOGON@C754,ANONYMOUS LOGON@C754,NTLM,Network,LogOn,Success
2,3,C18769,C754,816792,C18769$@DOM1,C18769$@DOM1,NTLM,Network,LogOn,Success
3,4,C5808,C18781,1770007,ANONYMOUS LOGON@C18781,ANONYMOUS LOGON@C18781,NTLM,Network,LogOn,Success
4,5,C13050,C13050,30731,C13050$@DOM1,C13050$@DOM1,?,?,TGT,Success


In [3]:
# just considering the users in source_user@domain

users_df = df[df['source_user@domain'].str.contains("U")]
users_df = users_df[-users_df['source_user@domain'].str.contains("ANONYMOUS")]
users_df = users_df[['time', 'destination_computer', 'source_user@domain']]
users_df['source_user@domain'] = users_df['source_user@domain'].str.split('@').str.get(0)

users_df

,time,destination_computer,source_user@domain
9,1254860,C2805,U2552
10,24716,C1877,U2521
27,26627,C2327,U581
32,2981235,C2721,U4995
34,3238561,C1025,U5195
42,2359124,C2058,U2398
44,35124,C743,U6230
45,26344,C2327,U2973
46,726075,C2327,U6239
51,115497,C528,U3174


In [ ]:
#U6836, U1653, U7998

In [52]:
users_df1 = users_df[users_df['source_user@domain'].str.contains("U1723|U1653|U66")]
#users_df[users_df['source_user@domain'].str.contains("U66")]['destination_computer'].value_counts()

In [68]:
# Create a corpus, each element represents one day of connections 

sec_day = 86400
corpus = []
for i in range(57):
    day = (users_df1["time"] <= sec_day*(i+1)) & (users_df1["time"] > sec_day*i)
    list = users_df1[day][['destination_computer']]
    corpus.append(list)



In [69]:
# transform the dataframe into array
users_df_list=[]
for i in range(57):
    a = [x for xs in corpus[i].values for x in xs]
    users_df_list.append(a)




In [27]:
users_df_list

[['C612',
  'C4386',
  'C625',
  'C585',
  'C1940',
  'C2327',
  'C3075',
  'C2055',
  'C647',
  'C2803',
  'C457',
  'C3722',
  'C545',
  'C1759',
  'C2106',
  'C2368',
  'C1326',
  'C1877',
  'C1877',
  'C74',
  'C1153',
  'C2021',
  'C596',
  'C5519',
  'C864',
  'C479',
  'C1646',
  'C1823',
  'C585',
  'C1877',
  'C2062',
  'C2327',
  'C2327',
  'C2341',
  'C2327',
  'C529',
  'C585',
  'C1998',
  'C2327',
  'C3188',
  'C2525',
  'C5825',
  'C2553',
  'C1585',
  'C1212',
  'C1152',
  'C1790',
  'C2765',
  'C2490',
  'TGT',
  'C1877',
  'C307',
  'C2327',
  'C832',
  'C2737',
  'C1333',
  'C743',
  'C586',
  'C5722',
  'C4146',
  'C1102',
  'C3056',
  'C585',
  'C214',
  'C2553',
  'C2327',
  'C5716',
  'C1065',
  'C5725',
  'C2707',
  'C585',
  'C22',
  'C2553',
  'C3280',
  'C1640',
  'C2106',
  'C585',
  'C1622',
  'C2545',
  'C1056',
  'C5825',
  'C743',
  'C2327',
  'C2327',
  'C3547',
  'C2106',
  'C625',
  'C2553',
  'C586',
  'C3057',
  'C3055',
  'C1640',
  'C2706',
  'C23

In [70]:
#create corpus
id2word = corpora.Dictionary(users_df_list)

texts = users_df_list

corpus1 = [id2word.doc2bow(text) for text in texts]

corpus1

[[(0, 1),
  (1, 3),
  (2, 7),
  (3, 2),
  (4, 1),
  (5, 3),
  (6, 11),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 3),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 1),
  (50, 2),
  (51, 1),
  (52, 1),
  (53, 12),
  (54, 1),
  (55, 2),
  (56, 2),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 3),
  (62, 2),
  (63, 1),
  (64, 3),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 4),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 17),
  (78, 2),
  (79, 2),
  (80, 2),
  (81, 2),
  (82, 1),
  (83, 3),
  (84, 1),
  (85, 2),
  (86, 1),
  (87, 1),
  (88, 2),
  (89, 1),
  (90, 1),
  (91,

In [71]:
##lda model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus1,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [72]:
pprint(lda_model.print_topics())
#"U1723|U1653|U66"
# 2:U66 1: U1723 0:U1653 

[(0,
  '0.848*"C5666" + 0.018*"C1025" + 0.014*"C8799" + 0.003*"C549" + '
  '0.002*"C1877" + 0.002*"C585" + 0.001*"C148" + 0.001*"C423" + 0.001*"C2378" '
  '+ 0.001*"C2327"'),
 (1,
  '0.062*"C504" + 0.041*"C2651" + 0.039*"C754" + 0.034*"C549" + 0.030*"C1003" '
  '+ 0.027*"C1173" + 0.027*"C2173" + 0.027*"C1056" + 0.027*"C231" + '
  '0.024*"C3468"'),
 (2,
  '0.049*"C2327" + 0.043*"C585" + 0.039*"C743" + 0.033*"C1877" + 0.028*"C2553" '
  '+ 0.017*"C586" + 0.016*"C5716" + 0.016*"C1640" + 0.016*"C2106" + '
  '0.014*"C612"')]


In [74]:
users_df[users_df['source_user@domain'].str.contains("U66")]['destination_computer'].value_counts()

C2327     84
C585      83
C743      68
C1877     64
C1025     49
C2553     46
C8799     36
C612      27
C586      27
C423      26
C2106     26
C1640     26
C5716     25
C92       23
C1065     22
C625      21
C529      21
U7        20
C528      19
C457      19
C467      18
C2651     18
TGT       14
C1790     13
C1015     11
C2310     11
C2525     10
C449       9
C16818     8
C2162      8
          ..
C5699      1
C23848     1
C10335     1
C14992     1
C25825     1
C5726      1
C134       1
C2564      1
C22204     1
C15742     1
C3419      1
C9961      1
C2055      1
C2583      1
C626       1
C2925      1
C177       1
C2140      1
C19168     1
C24566     1
C3573      1
C755       1
C24185     1
C2725      1
C2069      1
C1706      1
C2196      1
C3709      1
C3373      1
C5264      1
Name: destination_computer, Length: 452, dtype: int64

In [75]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus1))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=users_df_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -2.861276872311476

Coherence Score:  0.584194934919811


In [76]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus1, id2word)
vis

C:\Users\seeml\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.336470 -0.003054       1        1  60.695259
2     -0.175276 -0.107927       2        1  27.082067
1     -0.161193  0.110981       3        1  12.222665, topic_info=     Category         Freq    Term        Total  loglift  logprob
term                                                             
269   Default  2533.000000   C5666  2533.000000  30.0000  30.0000
255   Default    48.000000    C504    48.000000  29.0000  29.0000
112   Default    73.000000   C2327    73.000000  28.0000  28.0000
293   Default    71.000000    C585    71.000000  27.0000  27.0000
308   Default    58.000000    C743    58.000000  26.0000  26.0000
140   Default    31.000000   C2651    31.000000  25.0000  25.0000
310   Default    29.000000    C754    29.000000  24.0000  24.0000
267   Default    33.000000    C549    33.000000  23.0000  23.0000
77    Default    55.000000   C1877    55.000000  22.0000  22.0000
133   Default    40.000000   C2553    40.000000  21.0000  21.0000
1     Default    25.000000   C1003    25.000000  20.0000  20.0000
101   Default    24.000000   C2173    24.000000  19.0000  19.0000
15    Default    23.000000   C1173    23.000000  18.0000  18.0000
5     Default    22.000000   C1056    22.000000  17.0000  17.0000
109   Default    21.000000    C231    21.000000  16.0000  16.0000
204   Default    22.000000   C3468    22.000000  15.0000  15.0000
304   Default    22.000000    C706    22.000000  14.0000  14.0000
274   Default    22.000000   C5716    22.000000  13.0000  13.0000
294   Default    24.000000    C586    24.000000  12.0000  12.0000
53    Default    23.000000   C1640    23.000000  11.0000  11.0000
93    Default    23.000000   C2106    23.000000  10.0000  10.0000
237   Default    21.000000    C423    21.000000   9.0000   9.0000
6     Default    20.000000   C1065    20.000000   8.0000   8.0000
298   Default    23.000000    C612    23.000000   7.0000   7.0000
577   Default    52.000000   C1025    52.000000   6.0000   6.0000
258   Default    16.000000    C528    16.000000   5.0000   5.0000
299   Default    18.000000    C625    18.000000   4.0000   4.0000
333   Default    17.000000      U7    17.000000   3.0000   3.0000
244   Default    17.000000    C457    17.000000   2.0000   2.0000
259   Default    18.000000    C529    18.000000   1.0000   1.0000
...       ...          ...     ...          ...      ...      ...
503    Topic3     1.000049   C7235     1.598430   1.6329  -6.3940
452    Topic3     1.000090    C243     1.598519   1.6329  -6.3940
502    Topic3     0.999878   C6754     1.598442   1.6327  -6.3942
522    Topic3     0.999727   C1484     1.598447   1.6326  -6.3944
460    Topic3     0.999808   C3813     1.598587   1.6326  -6.3943
431    Topic3     0.999793   C9302     1.598606   1.6325  -6.3943
267    Topic3    20.089735    C549    33.776546   1.5823  -3.3939
304    Topic3    13.257102    C706    22.147739   1.5887  -3.8096
110    Topic3     5.052755   C2310     9.592504   1.4608  -4.7742
337    Topic3     2.966009  C16818     5.604912   1.4655  -5.3069
131    Topic3     4.534990   C2525     9.695137   1.3421  -4.8823
430    Topic3     1.749511   C5865     3.205419   1.4964  -5.8348
237    Topic3     7.967181    C423    21.461014   1.1110  -4.3188
413    Topic3     1.740496  C16571     3.208533   1.4902  -5.8399
496    Topic3     1.748314   C1681     3.280863   1.4724  -5.8354
333    Topic3     6.285135      U7    17.354738   1.0862  -4.5559
342    Topic3     2.309215   C2331     4.839079   1.3621  -5.5572
346    Topic3     5.392764     C92    19.189867   0.8326  -4.7090
428    Topic3     1.870186  C13170     4.183500   1.2968  -5.7681
293    Topic3     8.902040    C585    71.018753   0.0252  -4.2078
245    Topic3     3.655967    C467    16.622505   0.5875  -5.0977
68     Topic3     3.022619   C1790    11.684703   0.7497  -5.2880
242    Topic3     2.578418    C449     8.377243   0.9235 